### Se obtinenen las imágenes y se almacenan
Las imágenes se descargan de la siguiente url : https://www.wikiart.org/en/paintings-by-style/

In [36]:
import requests
import json
from bs4 import BeautifulSoup
import pandas as pd
import time


from raven import Client
client = Client('https://6293821d2e964169aed64c863b9f5eb0@sentry.io/1473077')

In [ ]:
##TODO: pasar a archivo de configuración
#estilos = ['abstract-art', 'pop-art', 'realism', 'surrealism']

In [2]:
estilos = ['abstract-art', 'realism']
tipos = ['train', 'test', 'validar']

path_data = 'data/'
path_train = 'imagenes_color/train/'
path_test = 'imagenes_color/test/'
path_validar = 'imagenes_color/validar/'
path_training = 'training/'


pags_train = list(range(1, 11))
pags_test = list(range(11, 14))
pags_validar = list(range(20, 23))

In [3]:
def descarga_img(id_imagen, url, carpeta, tipo):
    '''
    descarga las imágenes
    a la carpeta especificada
    tipo : train ,test, validar
    '''
    
    path = 'imagenes_color/{}/{}/'.format(tipo, carpeta)
    try:
        nombre = str(id_imagen) + '-' + url.split('/')[-1]
        imagen = requests.get(url)

        with open(path + nombre, 'wb') as f:  
            f.write(imagen.content)  
        
    except Exception as e:
        client.captureException()
        return 'Error: ' + str(e)            

In [4]:
def datos_datos(estilos, pags):
    '''
    obtiene los json y los almacena 
    en una lista
    '''
    lista_json = []
    url = 'https://www.wikiart.org/en/paintings-by-style/'
    try:
        for e in estilos:
            for i in pags:
                url_json = '{}{}?json=2&page={}'.format(url, e, i)
                print(url_json)
                response = requests.get(url_json).json()

                for item in response['Paintings']:
                    item.update( {"estilo": e})

                lista_json.append(response['Paintings'])
                time.sleep(10 / 100)
                
    except Exception as e:
        client.captureException()
        return 'Error: ' + str(e)                
            
    return lista_json
    

In [5]:
def crea_df(lista_json):
    '''
    Devuelve un dataframe compuesto
    por la lista de json
    '''
    df = pd.DataFrame()
    for i in lista:
        df = df.append(i, ignore_index = True)
        
    return df

In [ ]:
def guardar_csv(data, title):
    data.to_csv( 'data/' + title + '.csv', index = False)   

In [ ]:
lista = datos_datos(estilos, pags)

In [86]:
df = crea_df(lista)
df.head()

,albums,artistName,artistUrl,estilo,flags,height,id,image,images,map,paintingUrl,title,width,year
0,None,Olowe of Ise,/en/olowe-of-ise,abstract-art,2,1783,5a5501bfedc2c95e08d31c28,https://uploads3.wikiart.org/00166/images/olow...,"[{'width': 750, 'height': 1220, 'image': 'http...",01234567*,/en/olowe-of-ise/lintel,Lintel,750,XIX cent.
1,None,Alphonse Allais,/en/alphonse-allais,abstract-art,2,848,57728a62edc2cb388010efa1,https://uploads0.wikiart.org/images/alphonse-a...,None,01234*67*,/en/alphonse-allais/first-communion-of-anaemic...,First Communion of Anaemic Young Girls in the ...,1324,1883
2,None,Alphonse Allais,/en/alphonse-allais,abstract-art,2,980,57728a61edc2cb388010ef5f,https://uploads2.wikiart.org/images/alphonse-a...,None,01234*67*,/en/alphonse-allais/apoplectic-cardinals-harve...,Apoplectic Cardinals Harvesting Tomatoes on th...,1400,1884
3,None,Alphonse Allais,/en/alphonse-allais,abstract-art,2,980,57728a61edc2cb388010ef71,https://uploads2.wikiart.org/images/alphonse-a...,None,01234*67*,/en/alphonse-allais/band-of-greyfriars-in-the-...,Band of Greyfriars in the Fog (Band Of Dusty D...,1400,1884
4,None,Alphonse Allais,/en/alphonse-allais,abstract-art,2,560,57728a61edc2cb388010ef81,https://uploads5.wikiart.org/images/alphonse-a...,None,0123**67*,/en/alphonse-allais/negroes-fighting-in-a-tunn...,Negroes Fighting in a Tunnel by Night,800,1884


In [87]:
df.shape

(2400, 14)

In [ ]:
guardar_csv(df, path_data + 'datos-train-raw.csv')

### Se descargan las imágenes

In [92]:
imagenes = df[['estilo', 'image']]

for e in range(0, imagenes.shape[0] ):
    descarga_img(e, 
                 imagenes.iloc[e]['image'], 
                 imagenes.iloc[e]['estilo'], 
                 'train')
    
    time.sleep(5 / 1000) # se duerme el bucle entre cada petición

https://uploads3.wikiart.org/00166/images/olowe-of-ise/an00525348-001-l-1.jpg
https://uploads0.wikiart.org/images/alphonse-allais/first-communion-of-anaemic-young-girls-in-the-snow.jpg
https://uploads2.wikiart.org/images/alphonse-allais/apoplectic-cardinals-harvesting-tomatoes-on-the-shore-of-the-red-sea-study-of-the-aurora.jpg
https://uploads2.wikiart.org/images/alphonse-allais/band-of-greyfriars-in-the-fog-band-of-dusty-drunks-in-the-fog.jpg
https://uploads5.wikiart.org/images/alphonse-allais/negroes-fighting-in-a-tunnel-by-night.jpg
https://uploads6.wikiart.org/images/alphonse-allais/fear-of-navy-recruits-seeing-the-mediterranean-for-the-first-time.jpg
https://uploads1.wikiart.org/images/alphonse-allais/jaundiced-cuckolds-handling-ochre.jpg
https://uploads2.wikiart.org/images/alphonse-allais/some-pimps-known-as-green-backs-on-their-bellies-in-the-grass-drinking-absinthe.jpg
https://uploads0.wikiart.org/images/hilma-af-klint/chaos-nr-2-1906.jpg
https://uploads7.wikiart.org/images/dav

https://uploads2.wikiart.org/images/marsden-hartley/abstraction-1914.jpg
https://uploads1.wikiart.org/images/marsden-hartley/portrait-of-a-german-officer-1914.jpg
https://uploads0.wikiart.org/images/david-bomberg/in-the-hold-1914.jpg
https://uploads5.wikiart.org/images/vanessa-bell/abstract-painting-1914.jpg
https://uploads1.wikiart.org/images/vanessa-bell/preliminary-design-for-lady-hamilton-rug-1914.jpg
https://uploads8.wikiart.org/images/jacoba-van-heemskerck/composition-1914.jpg
https://uploads5.wikiart.org/images/wassily-kandinsky/untitled.jpg
https://uploads6.wikiart.org/images/paul-klee/side-panels-for-anatomy-of-aphrodite-1915(1).jpg
https://uploads2.wikiart.org/images/henri-matisse/the-yellow-curtain.jpg
https://uploads0.wikiart.org/images/georgia-o-keeffe/special-no-32.jpg
https://uploads8.wikiart.org/images/jean-arp/abstract-composition.jpg
https://uploads8.wikiart.org/images/theo-van-doesburg/abstract-church-1915.jpg
https://uploads0.wikiart.org/images/johannes-itten/horizo

https://uploads5.wikiart.org/images/georgia-o-keeffe/blue-line.jpg
https://uploads2.wikiart.org/images/georgia-o-keeffe/from-the-plains.jpg
https://uploads4.wikiart.org/images/johannes-itten/linienrhythmus-1919.jpg
https://uploads1.wikiart.org/images/johannes-itten/komposition-1919.jpg
https://uploads1.wikiart.org/images/janos-mattis-teutsch/composition-1919(3).jpg
https://uploads1.wikiart.org/images/janos-mattis-teutsch/composition-1919(4).jpg
https://uploads1.wikiart.org/images/janos-mattis-teutsch/composition-1919(5).jpg
https://uploads3.wikiart.org/images/janos-mattis-teutsch/composition-1919-1(5).jpg
https://uploads0.wikiart.org/images/janos-mattis-teutsch/composition-1919-2.jpg
https://uploads0.wikiart.org/images/janos-mattis-teutsch/composition-1919-3.jpg
https://uploads0.wikiart.org/images/janos-mattis-teutsch/composition-1919-4.jpg
https://uploads0.wikiart.org/images/janos-mattis-teutsch/composition-1919-5.jpg
https://uploads1.wikiart.org/images/joaquin-torres-garcia/el-puerto

https://uploads1.wikiart.org/images/wassily-kandinsky/green-composition-1923.jpg
https://uploads4.wikiart.org/images/wassily-kandinsky/on-white-ii-1923.jpg
https://uploads8.wikiart.org/images/wassily-kandinsky/orange-1923.jpg
https://uploads8.wikiart.org/images/wassily-kandinsky/transverse-line-1923.jpg
https://uploads5.wikiart.org/images/georgia-o-keeffe/gray-line-with-black-blue-and-yellow.jpg
https://uploads8.wikiart.org/images/constantin-brancusi/bird-in-space-1923.jpg
https://uploads1.wikiart.org/images/constantin-brancusi/bird-in-space-1923-1.jpg
https://uploads1.wikiart.org/images/paul-klee/architecture-of-the-plain-1923.jpg
https://uploads1.wikiart.org/images/janos-mattis-teutsch/composition-1923(4).jpg
https://uploads1.wikiart.org/images/janos-mattis-teutsch/composition-1923(5).jpg
https://uploads1.wikiart.org/images/janos-mattis-teutsch/composition-1923(6).jpg
https://uploads3.wikiart.org/images/janos-mattis-teutsch/composition-1923-1(8).jpg
https://uploads7.wikiart.org/image

https://uploads2.wikiart.org/images/david-kakabadze/composition-1926.jpg
https://uploads7.wikiart.org/images/wassily-kandinsky/dark-freshness-1927.jpg
https://uploads8.wikiart.org/images/paul-klee/cote-de-provence(1).jpg
https://uploads1.wikiart.org/images/paul-klee/flora-on-sand-1927(1).jpg
https://uploads4.wikiart.org/images/paul-klee/pastoral-rhythms-1927(1).jpg
https://uploads7.wikiart.org/images/salvador-dali/harlequin-1927.jpg
https://uploads8.wikiart.org/images/salvador-dali/head-of-a-woman.jpg
https://uploads0.wikiart.org/images/georgia-o-keeffe/abstraction-white-rose.jpg
https://uploads7.wikiart.org/images/josef-ma/vejce-1927.jpg
https://uploads3.wikiart.org/images/jean-fautrier/le-lilas-blanc-1927.jpg
https://uploads1.wikiart.org/images/auguste-herbin/composition-1927.jpg
https://uploads5.wikiart.org/images/serge-charchoune/grossissement-1927.jpg
https://uploads4.wikiart.org/images/serge-charchoune/the-small-black-holes-1927.jpg
https://uploads4.wikiart.org/images/enrico-pram

...

https://uploads3.wikiart.org/images/max-ernst/the-wavering-woman-1923.jpg
https://uploads1.wikiart.org/images/max-ernst/woman-old-man-and-flower-femme.jpg
https://uploads7.wikiart.org/images/georges-papazoff/autoportrait-avec-madame-papazoff-1923.jpg
https://uploads3.wikiart.org/images/andre-masson/landscape-with-rocks.jpg
https://uploads6.wikiart.org/images/edward-wadsworth/la-rochelle-1923(1).jpg
https://uploads8.wikiart.org/images/edward-wadsworth/seaport-1923.jpg
https://uploads4.wikiart.org/images/leon-underwood/torso-1923.jpg
https://uploads0.wikiart.org/images/xul-solar/juzgue-1923.jpg
https://uploads3.wikiart.org/images/xul-solar/a-di-fana-1923.jpg
https://uploads0.wikiart.org/images/xul-solar/t-y-yo-1923.jpg
https://uploads5.wikiart.org/images/paul-klee/botanical-theater-1924(1).jpg
https://uploads8.wikiart.org/images/salvador-dali/portrait-of-garcia-lorca.jpg
https://uploads0.wikiart.org/images/pablo-picasso/guitar-glass-and-fruit-dish-with-fruit-1924.jpg
https://uploads3.wik

https://uploads5.wikiart.org/images/rene-magritte/the-difficult-crossing-1926(1).jpg
https://uploads1.wikiart.org/images/rene-magritte/the-face-of-genius-1926(1).jpg
https://uploads7.wikiart.org/images/rene-magritte/the-lost-jockey-1926(1).jpg
https://uploads8.wikiart.org/images/rene-magritte/the-musings-of-the-solitary-walker-1926(1).jpg
https://uploads3.wikiart.org/images/rene-magritte/the-sensational-news-1926(1).jpg
https://uploads6.wikiart.org/images/rene-magritte/the-silver-gap-1926(1).jpg
https://uploads2.wikiart.org/images/rene-magritte/threshold-of-forest-1926(1).jpg
https://uploads0.wikiart.org/images/rene-magritte/untitled-1926(1).jpg
https://uploads3.wikiart.org/images/rene-magritte/untitled-collage(1).jpg
https://uploads6.wikiart.org/images/rene-magritte/wreackage-of-the-shadow-1926(1).jpg
https://uploads2.wikiart.org/images/salvador-dali/honey-is-sweeter-than-blood.jpg
https://uploads0.wikiart.org/images/pablo-picasso/artist-and-his-model-1926.jpg
https://uploads3.wikiart

https://uploads3.wikiart.org/images/edward-weston/shell-1927.jpg
https://uploads6.wikiart.org/images/edward-weston/shell-1927(1).jpg
https://uploads6.wikiart.org/images/victor-brauner/portrait-of-claude-sernet-1927.jpg
https://uploads0.wikiart.org/images/leon-arthur-tutundjian/composition-1927.jpg
https://uploads1.wikiart.org/images/leon-arthur-tutundjian/composition-abstraite-1927.jpg
https://uploads3.wikiart.org/images/leon-arthur-tutundjian/composition-cellulaire-au-cercle-rouge-1927(1).jpg
https://uploads6.wikiart.org/images/edward-wadsworth/gasteropoda-1927(1).jpg
https://uploads7.wikiart.org/images/joan-miro/painting-head-1927.jpg
https://uploads2.wikiart.org/images/joan-miro/painting-1927.jpg
https://uploads1.wikiart.org/images/pierre-roy/le-grand-bateau-1927.jpg
https://uploads8.wikiart.org/images/xul-solar/drago-1927.jpg
https://uploads5.wikiart.org/images/xul-solar/puerto-azul-1927.jpg
https://uploads4.wikiart.org/00219/images/maruja-mallo/desnudo-surrealista-1927.jpg
https:/

https://uploads4.wikiart.org/images/rene-magritte/the-giantess-1929(1).jpg
https://uploads0.wikiart.org/images/rene-magritte/the-living-mirror-1929(1).jpg
https://uploads7.wikiart.org/images/rene-magritte/the-treachery-of-images-this-is-not-a-pipe-1948(2).jpg
https://uploads4.wikiart.org/images/rene-magritte/the-tree-of-knowledge-1929(1).jpg
https://uploads2.wikiart.org/images/rene-magritte/threatening-weather-1929(1).jpg
https://uploads8.wikiart.org/images/salvador-dali/accommodations-of-desire.jpg
https://uploads5.wikiart.org/images/salvador-dali/desecration-descripti.jpg
https://uploads3.wikiart.org/images/salvador-dali/illumined-pleasures.jpg
https://uploads0.wikiart.org/images/salvador-dali/imperial-monument-of-woma-child-gala-utopian-fantasy.jpg
https://uploads2.wikiart.org/images/salvador-dali/invisible-man-study-for-the-painting.jpg
https://uploads3.wikiart.org/images/salvador-dali/playing-in-the-dark.jpg
https://uploads7.wikiart.org/images/salvador-dali/portrait-of-paul-eluard

https://uploads8.wikiart.org/images/brassai/untitled-from-paris-after-midnight-1930.jpg
https://uploads0.wikiart.org/images/kansuke-yamamoto/untitled-1930.jpg
https://uploads2.wikiart.org/images/kansuke-yamamoto/untitled-1930-1.jpg
https://uploads2.wikiart.org/images/kansuke-yamamoto/untitled-1930-2.jpg
https://uploads2.wikiart.org/images/kansuke-yamamoto/untitled-1930-3.jpg
https://uploads2.wikiart.org/images/kansuke-yamamoto/untitled-1930-4.jpg
https://uploads2.wikiart.org/images/kansuke-yamamoto/untitled-1930-5.jpg
https://uploads2.wikiart.org/images/kansuke-yamamoto/untitled-1930-6.jpg
https://uploads2.wikiart.org/images/kansuke-yamamoto/untitled-1930-7.jpg
https://uploads2.wikiart.org/images/kansuke-yamamoto/untitled-1930-8.jpg
https://uploads7.wikiart.org/images/abidin-dino/bal-k-pazar-1930.jpg
https://uploads0.wikiart.org/images/leon-arthur-tutundjian/adam-et-eve-1930.jpg
https://uploads8.wikiart.org/images/angel-planells/crimen-perfecto-1930(1).jpg
https://uploads3.wikiart.org/